Temporal Innovator 

In [ ]:
#%pip install mesa
#pip install ipykernel
#pip install ipywidgets --upgrade

In [2]:
from mesa import Agent, Model
from mesa.space import MultiGrid
import random

Initial setup:

In [4]:
class TimeAgent(Agent):
    def __init__(self, model):
        super().__init__(model) # Mesa 3: only model is passed
        self.model = model  # Explicitly store model reference if needed

    def step(self):
        # For Steps 1-5, ensure no collisions (step_count 0-4 internally)
        if self.model.step_count < 5:
            attempts = 0
            max_attempts = 10  # Prevent infinite loops
            while attempts < max_attempts:
                x, y = self.pos
                move = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)]) # Move randomly: up, down, left, right
                new_pos = (x + move[0], y + move[1])
                if (0 <= new_pos[0] < self.model.grid.width) and (0 <= new_pos[1] < self.model.grid.height):
                    if new_pos not in self.model.occupied_positions or new_pos == self.pos:
                        self.model.grid.move_agent(self, new_pos)
                        self.model.occupied_positions.discard(self.pos)
                        self.model.occupied_positions.add(new_pos)
                        break
                attempts += 1
        else:
            # After Step 5, allow normal movement
            x, y = self.pos
            move = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
            new_pos = (x + move[0], y + move[1])
            if (0 <= new_pos[0] < self.model.grid.width) and (0 <= new_pos[1] < self.model.grid.height):
                self.model.grid.move_agent(self, new_pos)

          
class TimeModel(Model):
    def __init__(self):
        super().__init__()
        self.grid = MultiGrid(10, 10, False)  # 10x10 grid, torus disabled
        self.schedule = []  # Manual agent list
        self.random = random.Random()
        self.step_count = 0  # Track step number
        self.occupied_positions = set()  # Track occupied positions
        
        # Create 3 agents, all starting at (0, 0)
        for _ in range(3):
            agent = TimeAgent(self)
            self.grid.place_agent(agent, (0, 0))  # Explicitly start at (0, 0)
            self.schedule.append(agent)
        self.occupied_positions.add((0, 0))  # Initially, only (0, 0) is occupied    

    def step(self):
        # Reset occupied positions for this step
        self.occupied_positions.clear()
        for agent in self.schedule:
            self.occupied_positions.add(agent.pos)
        
        random.shuffle(self.schedule)  # Random activation
        for agent in self.schedule:
            agent.step()
        self.step_count += 1  # Increment before printing    
        self.print_positions()
        

    # Print current positions of all agents
    def print_positions(self):
        print(f"Step {self.step_count}:")
        # Sort agents by unique_id for consistent print order
        sorted_agents = sorted(self.schedule, key=lambda a: a.unique_id)
        for agent in sorted_agents:
            print(f"Agent {agent.unique_id} at position {agent.pos}")

# Run the model
model = TimeModel()
# Print initial state as Step 0
print("Initial Place:")
sorted_agents = sorted(model.schedule, key=lambda a: a.unique_id)
for agent in sorted_agents:
    print(f"Agent {agent.unique_id} at position {agent.pos}")
# Run 5 additional steps (labeled 1 to 5)
for _ in range(5):
    model.step()


Initial Place:
Agent 1 at position (0, 0)
Agent 2 at position (0, 0)
Agent 3 at position (0, 0)
Step 1:
Agent 1 at position (0, 0)
Agent 2 at position (0, 1)
Agent 3 at position (1, 0)
Step 2:
Agent 1 at position (0, 0)
Agent 2 at position (1, 1)
Agent 3 at position (2, 0)
Step 3:
Agent 1 at position (1, 0)
Agent 2 at position (0, 1)
Agent 3 at position (3, 0)
Step 4:
Agent 1 at position (2, 0)
Agent 2 at position (0, 2)
Agent 3 at position (3, 1)
Step 5:
Agent 1 at position (1, 0)
Agent 2 at position (1, 2)
Agent 3 at position (3, 0)
